In [1]:
import pandas as pd
from sqlalchemy.orm import Session

In [2]:
pegawai = pd.read_csv("seed/DataPegawai.csv")

In [3]:
pegawai.tail()

,IdPegawai,KdJenisPegawai,KdTitle,NamaLengkap,NamaKeluarga,NamaPanggilan,JenisKelamin,TempatLahir,TglLahir,TglMasuk,TglKeluar
761,L001000068,7,4.0,IRKHAM ABDULLAH AZZAM,NaN,NaN,L,Tegal,7/8/1992 00:00:00,NaN,NaN
762,P023000575,23,2.0,"Santoso, Amd",NaN,NaN,NaN,NaN,NaN,NaN,NaN
763,P023000576,23,2.0,"Ayub, Amd",NaN,NaN,NaN,NaN,NaN,NaN,NaN
764,P023000577,23,2.0,"Dwi Setyo, Amd",NaN,NaN,NaN,NaN,NaN,NaN,NaN
765,P023000578,23,2.0,"Trisnanto, Amd",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
pgw_new = pegawai.drop(["KdJenisPegawai", "KdTitle", "TglMasuk", "TglKeluar"], axis=1)

In [35]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker


# SQLALCHEMY_DATABASE_URL = "mysql://root:@localhost:3306/codeburst"
SQLALCHEMY_DATABASE_URL = "postgresql://root:1@localhost:5432/kardinahku" #+"?gssencmode=disable"

engine = create_engine(
    SQLALCHEMY_DATABASE_URL,
)

SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

Base = declarative_base()


In [37]:
from sqlalchemy import Column, Integer, String, Text, Boolean, ForeignKey, DateTime, CHAR
from datetime import datetime
from sqlalchemy.orm import relationship

class User(Base):
    __tablename__ = "users"

    id = Column(Integer, primary_key=True, index=True)
    username = Column(String(50), unique=True)
    password = Column(String)
    email = Column(String, unique=True)
    is_active = Column(Boolean, default=False)
    role = Column(String, default='user')
    id_pegawai = Column(String, ForeignKey("pegawai.id"))
    token = Column(String, nullable=True)

    created_at = Column(DateTime, default=datetime.now)
    update_at = Column(DateTime, default=datetime.now, onupdate=datetime.now)
    deleted_at = Column(DateTime, nullable=True)

    pegawai = relationship("Pegawai", back_populates="user")
    masalah_lv1 = relationship("Masalah", back_populates="disposisi_1")
    masalah_lv2 = relationship("Masalah", back_populates="disposisi_2")
    masalah_lv3 = relationship("Masalah", back_populates="disposisi_3")


class Pegawai(Base):
    __tablename__ = "pegawai"

    id = Column(String(11), primary_key=True, index=True)
    nama_lengkap = Column(String(60))
    nama_panggilan = Column(String(60), nullable=True)
    tempat_lahir = Column(String(20), nullable=True)
    tanggal_lahir = Column(DateTime, nullable=True)
    jenis_kelamin = Column(CHAR(1), default='L')
    no_hp = Column(String(16), nullable=True)
    no_wa = Column(String(16), nullable=True)

    created_at = Column(DateTime, default=datetime.now)
    update_at = Column(DateTime, default=datetime.now, onupdate=datetime.now)
    deleted_at = Column(DateTime, nullable=True)

    user = relationship("User", back_populates="pegawai")


class KategoriMasalah(Base):
    __tablename__ = "kategori_masalah"

    id = Column(Integer, primary_key=True, index=True)
    kategori = Column(String(60))

    masalah = relationship("Masalah", back_populates="kategori_masalah")

class KategoriTindakan(Base):
    __tablename__ = "kategori_tindakan"

    id = Column(Integer, primary_key=True, index=True)
    kategori = Column(String(50))

    tindakan = relationship("Tindakan", back_populates="kategori_tindakan")


class Instalasi(Base):
    __tablename__ = "instalasi"

    id = Column(Integer, primary_key=True, index=True)
    nama = Column(String(40))

    ruangan = relationship("Ruangan", back_populates="instalasi")


class Ruangan(Base):
    __tablename__ = "ruangan"

    id = Column(Integer, primary_key=True, index=True)
    nama = Column(String(60))
    id_instalasi = Column(Integer, ForeignKey("instalasi.id"))

    instalasi = relationship("Instalasi", back_populates="ruangan")
    masalah = relationship("Masalah", back_populates="ruangan")
    sarana = relationship("Sarana", back_populates="ruangan")
    tindakan = relationship("Tindakan", back_populates="ruangan")

class JenisSarana(Base):
    __tablename__ = "jenis_sarana"

    id = Column(Integer, primary_key=True, index=True)
    nama = Column(String(30))

    created_at = Column(DateTime, default=datetime.now)
    update_at = Column(DateTime, default=datetime.now, onupdate=datetime.now)
    done_at = Column(DateTime, nullable=True)
    deleted_at = Column(DateTime, nullable=True)

    sarana = relationship("Sarana", back_populates="jenis")


class Sarana(Base):
    __tablename__ = "sarana"

    id = Column(Integer, primary_key=True, index=True)
    nama = Column(String(90))
    berat = Column(String(30), nullable=True)
    panjang = Column(String(30), nullable=True)
    lebar = Column(String(30), nullable=True)
    tinggi = Column(String(30), nullable=True)
    id_ruangan = Column(Integer, ForeignKey("ruangan.id"), nullable=True)
    id_jenis = Column(Integer, ForeignKey("jenis_sarana.id"))

    created_at = Column(DateTime, default=datetime.now)
    update_at = Column(DateTime, default=datetime.now, onupdate=datetime.now)
    deleted_at = Column(DateTime, nullable=True)

    masalah = relationship("Masalah", back_populates="sarana")
    ruangan = relationship("Ruangan", back_populates="sarana")
    tindakan = relationship("Tindakan", back_populates="sarana")
    jenis = relationship("JenisSarana", back_populates="sarana")
    # id_kamar = Column(Integer, ForeignKey(""))



class Masalah(Base):
    __tablename__ = "masalah"

    id = Column(Integer, primary_key=True, index=True)
    deskripsi = Column(Text)
    id_user = Column(Integer, ForeignKey("users.id"))
    id_kategori_masalah = Column(Integer, ForeignKey("kategori_masalah.id"))
    id_ruangan = Column(Integer, ForeignKey("ruangan.id"))
    id_sarana = Column(Integer, ForeignKey("sarana.id"))
    # id_level_1 = Column(Integer, ForeignKey("users.id"), nullable=True)
    # id_level_2 = Column(Integer, ForeignKey("users.id"), nullable=True)
    # id_level_3 = Column(Integer, ForeignKey("users.id"), nullable=True)
    status = Column(Boolean, default=False)
    foto = Column(String, nullable=True)


    created_at = Column(DateTime, default=datetime.now)
    update_at = Column(DateTime, default=datetime.now, onupdate=datetime.now)
    done_at = Column(DateTime, nullable=True)
    deleted_at = Column(DateTime, nullable=True)

    kategori_masalah = relationship("KategoriMasalah", back_populates="masalah")
    ruangan = relationship("Ruangan", back_populates="masalah")
    sarana = relationship("Sarana", back_populates="masalah")
    tindakan = relationship("Tindakan", back_populates="masalah")
    disposisi_1 = relationship("User", back_populates="masalah_lv1")
    disposisi_2 = relationship("User", back_populates="masalah_lv2")
    disposisi_3 = relationship("User", back_populates="masalah_lv3")

class Tindakan(Base):
    __tablename__ = "tindakan"

    id = Column(Integer, primary_key=True, index=True)
    kondisi_awal = Column(String)
    tindakan = Column(String)
    id_masalah = Column(Integer, ForeignKey("masalah.id"))
    id_kategori = Column(Integer, ForeignKey("kategori_tindakan.id"), nullable=True)
    id_sarana = Column(Integer, ForeignKey("sarana.id"), nullable=True)
    id_ruangan = Column(Integer, ForeignKey("ruangan.id"))
    foto = Column(String, nullable=True)

    created_at = Column(DateTime, default=datetime.now)
    update_at = Column(DateTime, default=datetime.now, onupdate=datetime.now)
    done_at = Column(DateTime, nullable=True)
    deleted_at = Column(DateTime, nullable=True)

    kategori_tindakan = relationship("KategoriTindakan", back_populates="tindakan")
    sarana = relationship("Sarana", back_populates="tindakan")
    ruangan = relationship("Ruangan", back_populates="tindakan")
    masalah = relationship("Masalah", back_populates="tindakan")


In [45]:
from sqlalchemy.orm import Session
import bcrypt
from datetime import datetime, timedelta
import numpy as np

In [39]:
db = SessionLocal()

In [40]:
ss = db.query(Ruangan).all()

In [55]:
pgw_new = pgw_new.astype(object).where(pd.notnull(pgw_new), None)
pgw_new.head()

,IdPegawai,NamaLengkap,NamaKeluarga,NamaPanggilan,JenisKelamin,TempatLahir,TglLahir
0,8888888888,Staff/Karyawan,None,None,L,None,1/1/1900 00:00:00
1,L013000001,"dr. WASRIL WAZAR, SpJp",None,None,L,Jakarta,23/4/1955 00:00:00
2,P013000002,dr. INDRIANA BUDIONO,None,None,P,Tegal,28/11/1954 00:00:00
3,L013000003,"dr. IZZUDIN DS, Sp.KJ.M.Kes",None,None,L,Pekalongan,15/10/1952 00:00:00
4,L013000004,"dr. ABDAL HAKIM TOHARI, SpRM.MMR",None,None,L,Nganjuk,6/7/1958 00:00:00


In [56]:
for i in range(0, pgw_new.shape[0]):
    pegawai = Pegawai(id=pgw_new.iloc[i]['IdPegawai'],
                      nama_lengkap=pgw_new.iloc[i]['NamaLengkap'],
                      nama_panggilan=pgw_new.iloc[i]['NamaPanggilan'],
                      jenis_kelamin=pgw_new.iloc[i]['JenisKelamin'],
                      tempat_lahir=pgw_new.iloc[i]['TempatLahir'],
                      tanggal_lahir=pgw_new.iloc[i]['TglLahir']
                     )
    db.add(pegawai)
    db.commit()
    db.refresh(pegawai)

InvalidRequestError: This Session's transaction has been rolled back due to a previous exception during flush. To begin a new transaction with this Session, first issue Session.rollback(). Original exception was: (psycopg2.errors.DatatypeMismatch) column "tanggal_lahir" is of type timestamp without time zone but expression is of type double precision
LINE 1: ...ANI, Sp.P', '19771019 200604 2 015', 'Surakarta', 'NaN'::flo...
                                                             ^
HINT:  You will need to rewrite or cast the expression.

[SQL: INSERT INTO pegawai (id, nama_lengkap, nama_panggilan, tempat_lahir, tanggal_lahir, jenis_kelamin, no_hp, no_wa, created_at, update_at, deleted_at) VALUES (%(id)s, %(nama_lengkap)s, %(nama_panggilan)s, %(tempat_lahir)s, %(tanggal_lahir)s, %(jenis_kelamin)s, %(no_hp)s, %(no_wa)s, %(created_at)s, %(update_at)s, %(deleted_at)s)]
[parameters: {'id': 'P013000096', 'nama_lengkap': 'dr. RENI ARI MARTANI, Sp.P', 'nama_panggilan': '19771019 200604 2 0

In [54]:
pgw_new.iloc[2]['IdPegawai']

'P013000002'